In [ ]:
import os
import sys

# Standardize working directory to project root
try:
    if os.path.basename(os.getcwd()) in ['modeling', 'scraping', 'data-prep', 'analysis', 'setup', 'misc']:
        os.chdir('../..')
    elif os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')
        
    if os.getcwd() not in sys.path:
        sys.path.append(os.getcwd())
        
    print(f"Working Directory set to: {os.getcwd()}")
except Exception as e:
    print(f"Failed to set working directory: {e}")

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [5]:
import mynnlib
from mynnlib import *
import imagehash

# Remove duplicate images

In [11]:
def clean_datasets(species_types, dataset_type='data'):
    for species_type in species_types:
        dataset_dir = f"insect-dataset/{species_type}/{dataset_type}"
        removed_corrupted = 0
        removed_duplicate = 0
        for class_name in os.listdir(dataset_dir):
            img_hashes = []
            for file in os.listdir(f"{dataset_dir}/{class_name}"):
                if not Path(f"{dataset_dir}/{class_name}/{file}").is_file():
                    removed_corrupted += 1
                    shutil.rmtree(f"{dataset_dir}/{class_name}/{file}")
                    continue
                img = Image.open(f"{dataset_dir}/{class_name}/{file}").convert("RGB")
                img_hash = imagehash.average_hash(img)
                if img_hash in img_hashes:
                    removed_duplicate += 1
                    os.remove(f"{dataset_dir}/{class_name}/{file}")
                    continue
                img_hashes += [img_hash]
        print(f"{dataset_dir:30}: removed_corrupted={removed_corrupted:5}, removed_duplicate={removed_duplicate:5}")

In [10]:
clean_datasets(["lepidoptera", "moth", "butterfly", "cicada", "odonata", "root-classifier"])

insect-dataset/lepidoptera/data: removed_corrupted=  433, removed_duplicate= 5496
insect-dataset/moth/data      : removed_corrupted=  330, removed_duplicate= 2884
insect-dataset/butterfly/data : removed_corrupted=   97, removed_duplicate= 2713
insect-dataset/cicada/data    : removed_corrupted=    0, removed_duplicate=  134
insect-dataset/odonata/data   : removed_corrupted=    0, removed_duplicate=  512
insect-dataset/root-classifier/data: removed_corrupted=    0, removed_duplicate= 1711


In [12]:
clean_datasets(["lepidoptera", "moth", "butterfly", "cicada", "odonata", "root-classifier"], dataset_type='val')

insect-dataset/lepidoptera/val: removed_corrupted=    0, removed_duplicate=    1
insect-dataset/moth/val       : removed_corrupted=    0, removed_duplicate=    0
insect-dataset/butterfly/val  : removed_corrupted=    0, removed_duplicate=    1
insect-dataset/cicada/val     : removed_corrupted=    0, removed_duplicate=    0
insect-dataset/odonata/val    : removed_corrupted=    0, removed_duplicate=    0
insect-dataset/root-classifier/val: removed_corrupted=    0, removed_duplicate=   17


# Remove backdated species

In [121]:
species = load_json("species.json")

In [22]:
def sort_species(species):
    species['lepidoptera']['butterfly'] = sorted(set(species['lepidoptera']['butterfly']))
    species['lepidoptera']['moth'] = sorted(set(species['lepidoptera']['moth']))
    species['odonata'] = sorted(set(species['odonata']))
    species['cicada'] = sorted(set(species['cicada']))
    return species

In [123]:
species['lepidoptera']['butterfly'] = list({re.sub(r"^(apharitis|spindasis)", "cigaritis", s) for s in species['lepidoptera']['butterfly']})
dump_json("species.json", sort_species(species))

# Find more species & data
India has almost ~12k moth and ~1.5k butterfly. Lepidoptera model is currently trained on 3739 species

In [2]:
species = load_json("species.json")

In [15]:
classes_with_data = {c for c in os.listdir(f"insect-dataset/lepidoptera/data") if not re.match(r"^.*-(early|genera|spp)$", c)}
lepidoptera_classes = set(species['lepidoptera']['moth'] + species['lepidoptera']['butterfly'])
print(f"total = {len(lepidoptera_classes)}")
no_data_classes = lepidoptera_classes - classes_with_data
print(f"with_data = {len(lepidoptera_classes - no_data_classes)}")
print(f"no_data = {len(no_data_classes)}")
print(f"\n{no_data_classes}")

total = 4175
with_data = 3947
no_data = 228

{'hasora-danda', 'pseudochazara-panjshira', 'darpa-minta', 'cacyparia-insolitata', 'aucha-latipennis', 'halpe-knyvetti', 'tapena-minuscula', 'halpe-molta', 'eulaceura-manipurensis', 'celaenorrhinus-flavocincta', 'halpe-perfossa', 'milleria-hamiltoni', 'karanasa-modesta', 'euthalia-thawgawa', 'parnassius-tianschanica', 'udara-selma', 'karanasa-leechi', 'pontia-glauconome', 'colias-leechi', 'halpe-arcuata', 'arhopala-zeta', 'polyommatus-hunza', 'shirozuozephyrus-birupa', 'deudorix-gaetulia', 'ochlodes-pasca', 'karanasa-astorica', 'cigaritis-acamas', 'graphium-adonarensis', 'karanasa-boloricus', 'euploea-roepstorffi', 'melitaea-lunulata', 'athyma-reta', 'argynnis-hyperbius-castetsi', 'orthomiella-ronkayana', 'chaetoprocta-kurumi', 'melitaea-shandura', 'hyponephele-neoza', 'allotinus-taras', 'chitoria-ulpi', 'ahlbergia-leechii', 'plebejus-rupala', 'hyponephele-tenuistigma', 'ypthima-lycus', 'potanthus-nina', 'boloria-hegemone', 'potanthus-alcon'

### Data from indianbiodiversity.org

In [16]:
moth_genus = set([ s.split('-')[0] for s in species['lepidoptera']['moth'] ])
moth_regex = f"^({'|'.join(moth_genus)})-.*"

butterfly_genus = set([ s.split('-')[0] for s in species['lepidoptera']['butterfly'] ])
butterfly_regex = f"^({'|'.join(butterfly_genus)})-.*"

ignore_regex = r"^(apharitis|spindasis).*"

In [20]:
# lepidoptera data extracted from indianbiodiversity.org

import json
from bs4 import BeautifulSoup
import Levenshtein

# with open("test.html", "r", encoding="utf-8") as file:
#     biodiversity_soup = BeautifulSoup(file, "html.parser")
# new_species = {i.get_text().lower().replace(" ", "-") for i in biodiversity_soup.find_all("i")}
new_species = {'mocis-frugalis', 'chiasmia-fidoniata', 'spindasis-lohita', 'caleta-elna', 'belenois-aurota', 'indomyrlaea-ferreotincta', 'eudocima-homaena', 'biston-suppressaria', 'scirpophaga-innotata', 'nausinoe-perspectata', 'chiasmia-hebesata', 'eurema-laeta', 'avatha-bubo', 'prosotas-bhutea', 'oxyodes-scrobiculata', 'glyphodes-bivitralis', 'theretra-suffusa', 'junonia-hierta', 'elymnias-caudata', 'charaxes-athamas', 'anthene-emolus', 'neptis-miah', 'phalanta-alcippe', 'tarucus-theophrastus', 'pareronia-valeria', 'callopistria-guttulalis', 'crithote-horridipes', 'papilio-helenus', 'agathodes-ostentalis', 'colotis-amata', 'bamra-mundata', 'papilio-demoleus', 'scobura-cephaloides', 'horaga-onyx', 'rapala-manea', 'hypena-laceratalis', 'areas-galactina', 'cnaphalocrocis-ruralis', 'glaucocharis-tripunctata', 'phaedyma-columella', 'heterostegane-subtessellata', 'agrotera-basinotata', 'clanis-phalaris', 'pingasa-ruginaria', 'creon-cleobis', 'lebadea-martha', 'flos-adriana', 'cirrochroa-thais', 'nepita-conferta', 'cyana-puella', 'delias-pasithoe', 'charana-mandarinus', 'lasippa-viraja', 'abisara-fylla', 'stibochiona-nicea', 'stauropus-alternus', 'prioneris-thestylis', 'ypthima-nareda', 'ariadne-merione', 'danaus-genutia', 'cimitra-seclusella', 'macroglossum-gyrans', 'callerebia-suroia', 'eutelia-blandiatrix', 'leptosia-nina', 'colotis-eucharis', 'theretra-nessus-nessus', 'agnosia-microta', 'traminda-mundissima', 'astictopterus-jama', 'erebus-ephesperis', 'timandra-convectaria', 'charltona-kala', 'arhopala-abseus', 'antheua-exanthemata', 'grammodes-geometrica', 'zesius-chrysomallus', 'spalgis-epeus', 'mycalesis-anaxias', 'eressa-confinis', 'ampittia-dioscorides', 'brahmaea-hearseyi', 'mimathyma-ambica', 'pieris-canidia', 'urapteroides-astheniata', 'charaxes-bernardus', 'abraxas-sylvata', 'loepa-katinka', 'amata-cyssea', 'asota-ficus', 'yepcalphis-dilectissima', 'papilio-paris', 'macaldenia-palumba', 'eumelea-ludovicata', 'kunugia-ampla', 'chiasmia-cymatodes', 'serrodes-campana', 'dalima-schistacearia', 'delias-acalis', 'chasmina-candida', 'lamproptera-curius', 'cechenena-helops', 'spindasis-syama', 'prioneris-clemanthe', 'eretmocera-impactella', 'metoeca-foedalis', 'charaxes-solon', 'cleora-injectaria', 'macrocilix-maia', 'herona-marathus', 'tagiades-japetus', 'iambrix-salsala', 'spialia-galba', 'aethaloessa-calidalis', 'callyna-costiplaga', 'neptis-clinia', 'maliattha-signifera', 'xestia', 'ischyja-manlia', 'graphium-agetes', 'colotis-etrida', 'cyrestis-thyodamas', 'orvasca-subnotota', 'megisba-malaya', 'tanaecia-julii', 'athyma-pravara', 'athyma-cama', 'amyna-punctum', 'pericyma-cruegeri', 'thalassodes-quadraria', 'pelagodes-falsaria', 'chalciope-mygdon', 'anomis', 'lexias-cyanipardus', 'charaxes-eudamippus', 'ophthalmitis-herbidaria', 'udara-dilecta', 'ischnurges-gratiosalis', 'acraea-terpsicore', 'cirrhochrista-brizoalis', 'parapoynx-crisonalis', 'elymnias-nesaea', 'mycalesis-malsarida', 'calesia-stillifera', 'euthalia-telchinia', 'mycalesis-gotama', 'talicada-nyseus', 'phalanta-phalantha', 'theretra-alecto', 'homaloxestis-myloxesta', 'lymantria-mathura', 'chiasmia-eleonora', 'prosotas-aluta', 'agrotis-ipsilon', 'delias-hyparete', 'palpita-annulifer', 'psimada-quadripennis', 'eurema-hecabe', 'papilio-castor', 'pycnarmon-cribrata', 'dodona-dipoea', 'sebastonyma-dolopia', 'trischalis-subaurana', 'athyma-perius', 'tagiades-gana', 'ancylolomia-saundersiella', 'pareronia-avatar', 'pleuroptya-balteata', 'mythimna-proleuca', 'cethosia-cyane', 'calliteara-grotei', 'dierna-acanthusalis', 'lethe-europa', 'papilio-paradoxa', 'zemeros-flegyas', 'neope-yama', 'hydrillodes-lentalis', 'cirrochroa-tyche', 'jamides-caeruleus', 'ypthima-ceylonica', 'chilades-pandava', 'caltoris-cahira', 'luxiaria-phyllosaria', 'neptis-jumbah', 'bassarona-teuta', 'eudocima-phalonia', 'speiredonia-mutabilis', 'tirumala-septentrionis', 'appias-albina', 'lemyra-spilosomata', 'streblote-dorsalis', 'delias-descombesi', 'barsine-cardinalis', 'yasoda-tripunctata', 'chilo-partellus', 'sphingomorpha-chlorea', 'eurema-andersonii', 'comostola-pyrrhogona', 'psilogramma-menephron', 'parotis-marginata', 'eudocima-hypermnestra', 'eublemma-parva', 'phazaca-leucocera', 'pericallia-matherana', 'kallima-horsfieldii', 'altha-subnotata', 'utetheisa-varians', 'chrysorabdia-bivitta', 'hydrillodes-gravatalis', 'aetholix-flavibasalis', 'endothenia-banausopis', 'prosotas-dubiosa', 'hippotion-rosetta', 'achaea-janata', 'lymantria-incerta', 'euthalia-lubentina', 'glyphodes-caesalis', 'euchromius-ocellea', 'earias-cupreoviridis', 'pyralis-pictalis', 'cyana-ridleyi', 'lophoptera-squammigera', 'athyma-inara', 'satarupa-splendens', 'rapala-pheretima', 'egnasia-ephyrodalis', 'pardomima-distorta', 'attatha-regalis', 'cepora-nadina', 'spoladea-recurvalis', 'helicoverpa-assulta', 'junonia-lemonias', 'pyrausta-panopealis', 'aloa-lactinea', 'hasora-badra', 'celastrina-lavendularis', 'hasora-chromus', 'trypanophora-semihyalina', 'tathorhynchus-exsiccata', 'callerebia-dibangensis', 'nyctemera-coleta', 'actias-selene', 'graphium-evemon', 'cnaphalocrocis-medinalis', 'graphium-nomius', 'leptotes-plinius', 'dysallacta-negatalis', 'papilio-nephelus', 'zizeeria-karsandra', 'macroglossum-mitchellii', 'chlumetia-transversa', 'ancylolomia-taprobanensis', 'symbrenthia-lilaea', 'graphium-macareus', 'zeheba-aureata', 'culladia-suffusella', 'theretra-oldenlandiae', 'ypthima-philomela', 'glyphodes-onychinalis', 'dercas-verhuelli', 'leptomiza-calcearia', 'acropteris-striataria', 'nausinoe-geometralis', 'teliphasa-albifusa', 'olene-mendosa', 'digama-hearseyana', 'bastilla-torrida', 'mangina-syringa', 'creatonotos-gangis', 'ramadasa-pavo', 'poritia-hewitsoni', 'euthalia-phemius', 'epiplema-reticulata', 'penicillaria-jocosatrix', 'utetheisa-lotrix', 'cephonodes-hylas', 'athyma-zeroca', 'zeltus-amasa', 'aporia-agathon', 'colias-croceus', 'euthalia-aconthea', 'cotachena-histricalis', 'pelopidas-conjuncta', 'omphisa-repetitalis', 'bastilla-crameri', 'anisoneura-aluco', 'junonia-iphita', 'koruthaialos-rubecula', 'telinga-malsara', 'lasiommata-schakra', 'cupha-erymanthis', 'sephisa-chandra', 'aglais-urticae', 'conogethes-punctiferalis', 'athyma-selenophora', 'isocentris-filalis', 'spodoptera-exigua', 'aegocera-bimacula', 'diasemiopsis-ramburialis', 'borbo-cinnara', 'eurema-brigitta', 'appias-lyncinda', 'athyma-ranga', 'nomophila-noctuella', 'polyura-athamas', 'phalera-grotei', 'sauris-eupitheciata', 'appias-olferna', 'chrysodeixis-acuta', 'chliaria-othona', 'euclasta-defamatalis', 'rapala-iarbus', 'scobura-isota', 'telicota-bambusae', 'feliniopsis-indistans', 'attacus-atlas', 'symbrenthia-hypselis', 'micronia-aculeata', 'badamia-exclamationis', 'sympis-rufibasis', 'agrius-convolvuli', 'xanthodes-malvae', 'comostola-laesaria', 'eudocima-salaminia', 'ericeia-pertendens', 'graphium-xenocles', 'caleta-decidia', 'anisephyra-ocularia', 'mocis-undata', 'samia-canningii', 'zizina-otis', 'conservula', 'curetis-thetis', 'jamides-bochus', 'polydesma-boarmioides', 'libythea-lepita', 'arhopala-fulla', 'acontia-nitidula', 'pachliopta-aristolochiae', 'lethe-chandica', 'junonia-almana', 'eutectona-machoeralis', 'nyctemera-lacticinia', 'syngamia-latimarginalis', 'conogethes-puntiferalis', 'heliophorus-brahma', 'celastrina-oreana', 'halpe-kusala', 'catapaecilma-major', 'arthroschista-hilaralis', 'arhopala-bazaloides', 'hellula-undalis', 'scobura-parawoolletti', 'antitrygodes-cuneilinea', 'aiteta-rufoflava', 'pseudozizeeria-maha', 'autocharis-fessalis', 'telicota-colon', 'endotricha-mesenterialis', 'helicoverpa-armigera', 'arhopala-perimuta', 'digama-marchalii', 'abisara-neophron', 'astygisa-albopunctata', 'ochus-subvittatus', 'rathinda-amor', 'tarucus-nara', 'zurobata-vacillans', 'leucophlebia-lineate', 'nannoarctia-dentata', 'callyna-jugaria', 'koruthaialos-butleri', 'arhopala-amantes', 'euploea-klugii', 'hypospila-bolinoides', 'colotis-fausta', 'zographetus-rama', 'banisia-myrsusalis', 'pieris-brassicae', 'satarupa-zulla', 'ypthima-chenui', 'notocrypta-paralysos', 'charaxes-marmax', 'atrophaneura-varuna', 'epiplema-himala', 'phycita-steniella', 'erebus-hieroglyphica', 'papilio-polytes', 'udea-ferrugalis', 'theretra-silhetensis-silhetensis', 'mycalesis-patnia', 'pelagodes-quadraria', 'nyctemera-adversata', 'pelopidas-assamensis', 'eustrotia-marginata', 'zamarada-excisa', 'neogurelca-hyas', 'botyodes-flavibasalis', 'macrobrochis-splendens', 'hypolycaena-othona', 'kaniska-canace', 'tanaecia-lepidea', 'utetheisa-pulchelloides', 'callopistria-maillardi', 'eilema-antica', 'graphium-eurypylus', 'acraea-issoria', 'tagiades-menaka', 'thyas-juno', 'acosmeryx-akanshi', 'phalera-combusta', 'filodes-fulvidorsalis', 'theretra-pallicosta', 'ticherra-acte', 'theretra-latreillii-lucasii', 'callerebia-hybrida', 'omiodes-indicata', 'erebus-caprimulgus', 'theretra-clotho', 'pelopidas-subochracea', 'mahathala-ameria', 'pantoporia-hordonia', 'asura-conferta', 'athetis-bremusa', 'homaea-clathrum', 'chadisra-bipars', 'delgamma-pangonia', 'ixias-marianne', 'creatonotos-transiens', 'dierna-strigata', 'lymantria-marginata', 'euploea-radamanthus', 'episteme-adulatrix', 'areas-imperialis', 'perina-nuda', 'virachola-isocrates', 'papilio-memnon', 'appias-libythea', 'eurrhyparodes-tricoloralis', 'theretra-silhetensis', 'jamides-alecto', 'potanthus-pseudomaesa', 'dichromia-pullata', 'vanessa-indica', 'grammodes-stolida', 'hulodes-caranea', 'gastropacha-pardale', 'orphanostigma-abruptalis', 'papilio-polymnestor', 'parasa-herbifera', 'tagiades-litigiosa', 'endotricha-consocia', 'etiella-zinckenella', 'dysphania-militaris', 'scopelodes-venosa', 'chersonesia-intermedia', 'amyna-selenampha', 'chiasmia-nora', 'udaspes-folus', 'argina-argus', 'hypsopygia-mauritialis', 'penthema-lisarda', 'acherontia-lachesis', 'pieris-melete', 'bastilla-arctotaenia', 'phazaca-theclata', 'nephele-hespera', 'amplypterus-panopus', 'lymantria-concolor', 'antheraea-mylitta', 'dodona-eugenes', 'delias-agostina', 'amblypodia-anita', 'hypolycaena-erylus', 'eupterote-mollifera', 'zizula-hylax', 'appias-lalage', 'neptis-yerburyi', 'macroglossum-stellatarum', 'cyclosia-papilionaris', 'halpe-zema', 'glyphodes-stolalis', 'lamprophaia-ablactalis', 'parthenos-sylvia', 'ctenoplusia-albostriata', 'amata-passalis', 'amblychia-hymenaria', 'athyma-kanwa', 'polytela-gloriosae', 'bombyx-mori', 'azanus-jesous', 'notocrypta-feisthamelii', 'parantica-melaneus', 'ophiusa-tirhaca', 'eublemma-anachoresis', 'graphium-chironides', 'jamides-celeno', 'spodoptera-littoralis', 'erebus-albicincta', 'serrodes-inara', 'pleuroptya-iopasalis', 'athetis-thoracica', 'azanus-ubaldus', 'hypena-lividalis', 'dichocrocis-rigidalis', 'arctioblepsis-rubida', 'nygmia-icilia', 'pindara-illibata', 'papilio-alcmenor', 'hebomoia-glaucippe', 'atrophaneura-aidoneus', 'charaxes-aristogiton', 'odontodes-aleuca', 'iton-semamora', 'danaus-chrysippus', 'eudocima-materna', 'protuliocnemis-biplagiata', 'barsine-striata', 'rhagastis-olivacea', 'aglais-rizana', 'leucophlebia-emittens', 'vanessa-cardui', 'noorda-blitealis', 'xenoplia', 'amraica-recursaria', 'chrysodeixis-eriosoma', 'alsophila-nilgirensis', 'theretra-nessus', 'aglaomorpha-plagiata', 'lampides-boeticus', 'artaxa-digramma', 'marumba-dyras', 'pyralis-manihotalis', 'cheritra-freja', 'lethe-verma', 'papilio-machaon', 'eurrhyparodes-bracteolalis', 'everes-diporides', 'problepsis-vulgaris', 'burara-amara', 'parnassius-hardwickii', 'azygophleps-pusilla', 'hyphormides-argentipunctata', 'sinna-extrema', 'surendra-quercetorum', 'remelana-jangala', 'rhinopalpa-polynice', 'delias-eucharis', 'lyclene-undulosa', 'celaenorrhinus-ambareesa', 'marumba-nympha', 'lyssa-zampa', 'syntypistis-fasciata', 'euripus-nyctelius', 'parasa-lepida', 'freyeria-trochylus', 'omiodes-analis', 'fascellina-plagiata', 'colotis-danae', 'pachynoa-sabelialis', 'euploea-algea', 'catopsilia-pyranthe', 'thysanoplusia-orichalcea', 'prosotas-nora', 'leucinodes-orbonalis', 'naranga-diffusa', 'tatobotys-janapalis', 'thaumantis-diores', 'pontia-daplidice', 'pirdana-major', 'neptis-namba', 'matapa-aria', 'rohana-parisatis', 'zographetus-satwa', 'earias-vitella', 'eublemma-dimidialis', 'lethe-mekara', 'spirama-helicina', 'macrobrochis-gigas', 'corma-maculata', 'acraea-violae', 'hyperythra-lutea', 'asota-plaginota', 'ragadia-crisilda', 'euthalia-durga', 'xyleutes-persona', 'pieris-rapae', 'zeuzera-indica', 'homodes-propitia', 'gangara-thyrsis', 'eupterote-lineosa', 'sumalia-daraxa', 'episparis-liturata', 'hypomecis-transcissa', 'aiteta-diurna', 'graphium-sarpedon', 'erebabraxas-metachromata', 'graphium-aristeus', 'taxila-haquinus', 'mycalesis-visala', 'hypolycaena-kina', 'talanga-sexpunctalis', 'catopsilia-pomona', 'ypthima-baldus', 'cydalima-laticostalis', 'orthocabera-ocernaria', 'trabala-vishnou', 'euthalia-anosia', 'ancistroides-nigrita', 'hypochrosis-hyadaria', 'artipe-eryx', 'theretra-castanea', 'netria-viridescens', 'ethope-himachala', 'paectes-subapicalis', 'netria-multispinae', 'westermannia-superba', 'oriens-goloides', 'epiplema', 'dophla-evelina', 'jamides-pura', 'colotis-vestalis', 'miresa-albipuncta', 'mota-massyla', 'vagrans-egista', 'euchromia-polymena', 'euploea-midamus', 'rhodometra-sacraria', 'xanthodes-intersepta', 'hesudra-divisa', 'pareronia-hippia', 'colias-fieldii', 'catochrysops-panormus', 'diplopseustis-periseresalis', 'scirpophaga-nepalensis', 'spirama-indenta', 'cosmophila-flava', 'tyspanodes-linealis', 'graphium-antiphates', 'papilio-clytia', 'arhopala-paramuta', 'telicota-ancilla', 'campylotes-histrionicus', 'mooreana-trichoneura', 'doleschallia-bisaltide', 'papilio-dravidarum', 'hyposidra-talaca', 'curetis-bulis', 'eudocima-cocalus', 'orsotriaena-medus', 'biston-contectaria', 'pygospila-octomaculalis', 'junonia-atlites', 'cupitha-purreea', 'cirrhochrista-fumipalpis', 'byblia-ilithyia', 'matapa-sasivarna', 'marumba-spectabilis', 'actias-maenas', 'heliophorus-epicles', 'satarupa-gopala', 'burara-gomata', 'aedia-acronyctoides', 'conolophia-nigripuncta', 'lethe-sinorix', 'colotis-aurora', 'trilocha-varians', 'bibasis-sena', 'maruca-amboinalis', 'junonia-orithya', 'parantica-pedonga', 'taractrocera-ceramas', 'lassaba-albidaria', 'erionota-torus', 'charaxes-delphis', 'artena-dotata', 'euthrix-laeta', 'thyas-coronata', 'parantica-aglea', 'nevrina-procopia', 'thyas-honesta', 'nacaduba-beroe', 'chersonesia-risa', 'eurema-andersoni', 'bradina-diagonalis', 'parapoynx-fluctuosalis', 'hydrillodes-transversalis', 'chasmina-cadida', 'neptis-pseudovikasi', 'scirpophaga-nivella', 'jamides-elpis', 'ypthima-huebneri', 'kallima-inachus', 'gonodontis-clelia', 'euthalia-monina', 'curetis-dentata', 'mythimna-loreyi', 'anthene-lycaenina', 'acontia-crocata', 'gadirtha-pulchra', 'hypocala-subsatura', 'sphragifera-rejecta', 'chiasmia-emersaria', 'parapoynx-stagnalis', 'catochrysops-strabo', 'sasunaga-tenebrosa', 'eublemma-cochylioides', 'acytolepis-puspa', 'argina-astrea', 'cirrhochrista-pulchellalis', 'bhutanitis-lidderdalii', 'asota-heliconia', 'euploea-core', 'pantoporia-sandaka', 'psilogramma-increta', 'metanastria-aconyta', 'charaxes-psaphon', 'anomis-privata', 'discrepans', 'papilio-mayo', 'chabula-acamasalis', 'bastilla-conficiens', 'lamprosema-tampiusalis', 'mangina-argus', 'agrotis-segetum', 'eurema-blanda', 'freyeria-putli', 'cepora-nerissa', 'suastus-gremius', 'spodoptera-litura', 'eublemma-baccalix', 'heliophorus-sena', 'xandrames-latiferaria', 'discolampa-ethion', 'marumba-indicus', 'herpetogramma-nigricornalis', 'ercheia-cyllaria', 'daphnis-nerii', 'artena-rubida', 'sameodes-cancellalis', 'rapala-varuna', 'amerila-astreus', 'hypocala-deflorata', 'enispe-euthymius', 'parantica-nilgiriensis', 'mycalesis-mineus', 'iraota-timoleon', 'agrotis-biconica', 'papilio-bianor', 'polygrammodes-pectinicornalis', 'araotes-lapithis', 'eoophyla-sejunctalis', 'dyrzela-plagiata', 'paliga-damastesalis', 'macroglossum-belis', 'maliarpha-separatella', 'gerosis-phisara', 'euploea-mulciber', 'pilocrocis-milvinalis', 'dichorragia-nesimachus', 'syntomoides-imaon', 'dysphania-percota', 'euchrysops-cnejus', 'amata-sperbius', 'troides-minos', 'papilio-buddha', 'dudusa-synopla', 'elymnias-malelas', 'problepsis-deliaria', 'hypena-thermesialis', 'oeonistis-entella', 'lestranicus-transpectus', 'polyptychus-dentatus', 'mycalesis-evansii', 'penicillaria-maculata', 'laelia-testacea', 'buzara-onelia', 'brevipecten-captatus', 'pygospila-tyres', 'rajendra-vittata', 'achaea-serva', 'oraesia-emarginata', 'neptis-soma', 'aeromachus-pygmaeus', 'hyblaea-puera', 'gandaca-harina', 'notocrypta-curvifascia', 'pachliopta-hector', 'eterusia-aedea', 'cirrhochrista-spissalis', 'nyctemera-arctata', 'phocoderma-velutina', 'hypolimnas-misippus', 'nausinoe-pueritia', 'macrobrochis-prasena', 'endotricha-melanobasis', 'cania-bilinea', 'asota-plana', 'lymantria-bivittata', 'darpa-pteria', 'terastia-egialealis', 'herpetogramma-cynaralis', 'aemene-taprobanis', 'didyma', 'ophiusa-indistincta', 'orthospila-orissusalis', 'arhopala-pseudocentaurus', 'parotis-nilgirica', 'pericyma-umbrina', 'moduza-procris', 'dysgonia-arcuata', 'hippotion-celerio', 'nebulosa', 'dysgonia-stuposa', 'gonepteryx-rhamni', 'scopula-pulchellata', 'agathia-lycaenaria', 'plusiodonta-coelonota', 'hemithea-tritonaria', 'agrioglypta-eurytusalis', 'plutodes-transmutata', 'coladenia-agni', 'pantoporia-paraka', 'bradina-melanoperas', 'euploea-doubledayi', 'thoressa-gupta', 'pithauria-stramineipennis', 'byasa-polyeuctes', 'ariadne-ariadne', 'fodina-oriolus', 'diaphania-indica', 'acherontia-styx', 'heliothis-irrorata', 'maruca-vitrata', 'curoba-sangarida', 'papilio-protenor', 'hypolimnas-bolina', 'chilades-lajus', 'celaenorrhinus-leucocera', 'erebus-macrops', 'heliophorus-moorei', 'charaxes-kahruba', 'bastilla-simillima', 'caligula-simla', 'pidorus-glaucopis', 'lethe-confusa', 'sarangesa-purendra', 'spindasis-vulcanus', 'catada-prominens', 'cyclodes-omma', 'tarucus-ananda', 'asota-sericea', 'neptis-harita', 'neptis-hylas', 'arsacia-rectalis', 'ixias-pyrene', 'parantica-sita', 'caltoris-kumara', 'arhopala-centaurus', 'symphaedra-nais', 'dichocrocis-zebralis', 'argynnis-hyperbius', 'omphisa-fuscidentalis', 'melanitis-leda', 'lethe-vindhya', 'idaea-macrospila', 'pelopidas-agna', 'melittia-chalciformis', 'antigastra-catalaunalis', 'olepa-ricini', 'hyles-livornica', 'choaspes-benjaminii', 'omphisa-anastomosalis', 'macaretaera-hesperis', 'deudorix-epijarbas', 'asota-producta', 'entomogramma-fautrix', 'hippoton-roseta', 'cethosia-nietneri', 'bastilla-joviana', 'leucinodes-apicalis', 'syllepte-sellalis', 'pagyda-salvalis', 'platyja-umminea', 'glyphodes-bicolor', 'loxura-atymnus', 'spodoptera-mauritia', 'mycalesis-francisca', 'spindasis-elima', 'odontoptilum-angulata', 'haritalodes-derogata', 'parotis-suralis', 'elymnias-hypermnestra', 'pergesa-acteus', 'vindula-erota', 'castalius-rosimon', 'hydriris-ornatalis', 'tajuria-cippus', 'rajendra-perrotteti', 'cethosia-biblis', 'botyodes-patulalis', 'appias-indra', 'tirumala-limniace', 'gerosis-sinica', 'crocidolomia-pavonana', 'achaea-mercatoria', 'rapala-nissa', 'amata-bicincta', 'trigonodes-regalis', 'ypthima-asterope', 'maruca-testulalis', 'orybina-plangonalis', 'pleurona-falcata', 'xanthodes-transversa', 'cydalima-conchylalis', 'aedia-leucomelas', 'blenina-donans', 'herpetogramma-bipunctalis', 'prioneris-sita', 'anuga-lunulata', 'celaenorrhinus-aurivittata', 'gonepteryx-amintha', 'istrugia-disputaria', 'pseudocoladenia-dan', 'mycalesis-perseus', 'arnetta-atkinsoni', 'neopithecops-zalmora', 'graphium-doson', 'sticopthalma-camadeva', 'papilio-krishna', 'rehimena-phrynealis', 'discophora-sondaica', 'orudiza-protheclaria', 'similis', 'abisara-chela', 'arctornis-comma', 'brithys-crini', 'pterothysanus-laticilia', 'cnaphalocrocis-poeyalis', 'trigonodes-disjuncta', 'anticarsia-irrorata', 'sinthusa-nasaka', 'hyarotis-adrastus', 'celaenorrhinus-ruficornis', 'ataboruza-divisa', 'hymenia-perspectalis', 'amyna-axis', 'taraka-hamada', 'delias-belladonna', 'trigonodes-hyppasia', 'thoressa-cerata', 'lymantria-similis', 'spirama-retorta', 'botyodes-asialis', 'sarangesa-dasahara', 'psolos-fuligo', 'lexias-dirtea', 'dacalana-penicilligera', 'heliothis-peltigera', 'graphium-agamemnon', 'asota-caricae', 'ceira-longipennis', 'gonepteryx-amintha-thibetana', 'hippotion-velox', 'lacera-procellosa', 'nagadeba-indecoralis', 'penicillaria-simplex', 'achrosis-rondelaria', 'carriola-ecnomoda', 'loboschiza-koenigiana', 'calyptra-minuticornis', 'appias-lyncida', 'papilio-crino', 'ophiusa-triphaenoides', 'hypopyra-vespertilio', 'cirrochroa-aoris', 'troides-helena', 'cyana-effracta', 'eressa-multigutta', 'maliattha-quadripartita', 'burara-oedipodea', 'eligma-narcissus', 'pseudergolis-wedah', 'synclera-traducalis', 'pannosa', 'oraesia-argyrosigna', 'telinga-malsarida', 'notarcha-quaternalis', 'faunis-canens'}
new_species = { s for s in new_species if len(s.split('-')) > 1 and not re.match(ignore_regex, s) }
new_species -= lepidoptera_classes

# discard possible typo
discarded_species = {s for s in new_species if min([ Levenshtein.distance(s, s2) for s2 in lepidoptera_classes ]) <= 2}
print(f"Discarded: {len(discarded_species)}")
print(discarded_species)
new_species -= discarded_species

new_moth = { s for s in new_species if re.match(moth_regex, s) }
print(f"\nMoth: {len(new_moth)}")
print(new_moth)

new_butterfly = { s for s in new_species if re.match(butterfly_regex, s) }
print(f"\nButterfly: {len(new_butterfly)}")
print(new_butterfly)

print(f"\nUnidentified: {len(new_species - new_moth - new_butterfly)}")
print(new_species - new_moth - new_butterfly)

Discarded: 19
{'chasmina-cadida', 'sticopthalma-camadeva', 'appias-lyncinda', 'leucophlebia-lineate', 'orvasca-subnotota', 'jamides-caeruleus', 'earias-vitella', 'hippoton-roseta', 'spalgis-epeus', 'ypthima-chenui', 'euthalia-durga', 'homodes-propitia', 'udara-dilecta', 'argina-argus', 'brevipecten-captatus', 'polydesma-boarmioides', 'conogethes-puntiferalis', 'eurema-andersoni', 'istrugia-disputaria'}

Moth: 104
{'bombyx-mori', 'eublemma-anachoresis', 'spodoptera-littoralis', 'serrodes-inara', 'athetis-thoracica', 'scirpophaga-innotata', 'hypena-lividalis', 'chiasmia-hebesata', 'avatha-bubo', 'barsine-striata', 'macroglossum-mitchellii', 'glyphodes-onychinalis', 'hypena-thermesialis', 'penicillaria-maculata', 'laelia-testacea', 'bastilla-torrida', 'rajendra-vittata', 'hyblaea-puera', 'cnaphalocrocis-ruralis', 'creatonotos-gangis', 'utetheisa-lotrix', 'nausinoe-pueritia', 'endotricha-melanobasis', 'cania-bilinea', 'syntypistis-fasciata', 'cotachena-histricalis', 'ophiusa-indistincta', 

In [55]:
species = load_json("species.json")
species['lepidoptera']['moth'] += new_moth
species['lepidoptera']['butterfly'] += new_butterfly
dump_json("species.json", sort_species(species))

In [56]:
# lepidoptera data extracted from indianbiodiversity.org

species = load_json("species.json")
lepidoptera_classes = set(species['lepidoptera']['moth'] + species['lepidoptera']['butterfly'])

# with open("test.html", "r", encoding="utf-8") as file:
#     biodiversity_soup = BeautifulSoup(file, "html.parser")
# new_species = {i.get_text().lower().replace(" ", "-") for i in biodiversity_soup.find_all("i")}
new_species = {'mocis-frugalis', 'chiasmia-fidoniata', 'spindasis-lohita', 'caleta-elna', 'belenois-aurota', 'indomyrlaea-ferreotincta', 'eudocima-homaena', 'biston-suppressaria', 'scirpophaga-innotata', 'nausinoe-perspectata', 'chiasmia-hebesata', 'eurema-laeta', 'avatha-bubo', 'prosotas-bhutea', 'oxyodes-scrobiculata', 'glyphodes-bivitralis', 'theretra-suffusa', 'junonia-hierta', 'elymnias-caudata', 'charaxes-athamas', 'anthene-emolus', 'neptis-miah', 'phalanta-alcippe', 'tarucus-theophrastus', 'pareronia-valeria', 'callopistria-guttulalis', 'crithote-horridipes', 'papilio-helenus', 'agathodes-ostentalis', 'colotis-amata', 'bamra-mundata', 'papilio-demoleus', 'scobura-cephaloides', 'horaga-onyx', 'rapala-manea', 'hypena-laceratalis', 'areas-galactina', 'cnaphalocrocis-ruralis', 'glaucocharis-tripunctata', 'phaedyma-columella', 'heterostegane-subtessellata', 'agrotera-basinotata', 'clanis-phalaris', 'pingasa-ruginaria', 'creon-cleobis', 'lebadea-martha', 'flos-adriana', 'cirrochroa-thais', 'nepita-conferta', 'cyana-puella', 'delias-pasithoe', 'charana-mandarinus', 'lasippa-viraja', 'abisara-fylla', 'stibochiona-nicea', 'stauropus-alternus', 'prioneris-thestylis', 'ypthima-nareda', 'ariadne-merione', 'danaus-genutia', 'cimitra-seclusella', 'macroglossum-gyrans', 'callerebia-suroia', 'eutelia-blandiatrix', 'leptosia-nina', 'colotis-eucharis', 'theretra-nessus-nessus', 'agnosia-microta', 'traminda-mundissima', 'astictopterus-jama', 'erebus-ephesperis', 'timandra-convectaria', 'charltona-kala', 'arhopala-abseus', 'antheua-exanthemata', 'grammodes-geometrica', 'zesius-chrysomallus', 'spalgis-epeus', 'mycalesis-anaxias', 'eressa-confinis', 'ampittia-dioscorides', 'brahmaea-hearseyi', 'mimathyma-ambica', 'pieris-canidia', 'urapteroides-astheniata', 'charaxes-bernardus', 'abraxas-sylvata', 'loepa-katinka', 'amata-cyssea', 'asota-ficus', 'yepcalphis-dilectissima', 'papilio-paris', 'macaldenia-palumba', 'eumelea-ludovicata', 'kunugia-ampla', 'chiasmia-cymatodes', 'serrodes-campana', 'dalima-schistacearia', 'delias-acalis', 'chasmina-candida', 'lamproptera-curius', 'cechenena-helops', 'spindasis-syama', 'prioneris-clemanthe', 'eretmocera-impactella', 'metoeca-foedalis', 'charaxes-solon', 'cleora-injectaria', 'macrocilix-maia', 'herona-marathus', 'tagiades-japetus', 'iambrix-salsala', 'spialia-galba', 'aethaloessa-calidalis', 'callyna-costiplaga', 'neptis-clinia', 'maliattha-signifera', 'xestia', 'ischyja-manlia', 'graphium-agetes', 'colotis-etrida', 'cyrestis-thyodamas', 'orvasca-subnotota', 'megisba-malaya', 'tanaecia-julii', 'athyma-pravara', 'athyma-cama', 'amyna-punctum', 'pericyma-cruegeri', 'thalassodes-quadraria', 'pelagodes-falsaria', 'chalciope-mygdon', 'anomis', 'lexias-cyanipardus', 'charaxes-eudamippus', 'ophthalmitis-herbidaria', 'udara-dilecta', 'ischnurges-gratiosalis', 'acraea-terpsicore', 'cirrhochrista-brizoalis', 'parapoynx-crisonalis', 'elymnias-nesaea', 'mycalesis-malsarida', 'calesia-stillifera', 'euthalia-telchinia', 'mycalesis-gotama', 'talicada-nyseus', 'phalanta-phalantha', 'theretra-alecto', 'homaloxestis-myloxesta', 'lymantria-mathura', 'chiasmia-eleonora', 'prosotas-aluta', 'agrotis-ipsilon', 'delias-hyparete', 'palpita-annulifer', 'psimada-quadripennis', 'eurema-hecabe', 'papilio-castor', 'pycnarmon-cribrata', 'dodona-dipoea', 'sebastonyma-dolopia', 'trischalis-subaurana', 'athyma-perius', 'tagiades-gana', 'ancylolomia-saundersiella', 'pareronia-avatar', 'pleuroptya-balteata', 'mythimna-proleuca', 'cethosia-cyane', 'calliteara-grotei', 'dierna-acanthusalis', 'lethe-europa', 'papilio-paradoxa', 'zemeros-flegyas', 'neope-yama', 'hydrillodes-lentalis', 'cirrochroa-tyche', 'jamides-caeruleus', 'ypthima-ceylonica', 'chilades-pandava', 'caltoris-cahira', 'luxiaria-phyllosaria', 'neptis-jumbah', 'bassarona-teuta', 'eudocima-phalonia', 'speiredonia-mutabilis', 'tirumala-septentrionis', 'appias-albina', 'lemyra-spilosomata', 'streblote-dorsalis', 'delias-descombesi', 'barsine-cardinalis', 'yasoda-tripunctata', 'chilo-partellus', 'sphingomorpha-chlorea', 'eurema-andersonii', 'comostola-pyrrhogona', 'psilogramma-menephron', 'parotis-marginata', 'eudocima-hypermnestra', 'eublemma-parva', 'phazaca-leucocera', 'pericallia-matherana', 'kallima-horsfieldii', 'altha-subnotata', 'utetheisa-varians', 'chrysorabdia-bivitta', 'hydrillodes-gravatalis', 'aetholix-flavibasalis', 'endothenia-banausopis', 'prosotas-dubiosa', 'hippotion-rosetta', 'achaea-janata', 'lymantria-incerta', 'euthalia-lubentina', 'glyphodes-caesalis', 'euchromius-ocellea', 'earias-cupreoviridis', 'pyralis-pictalis', 'cyana-ridleyi', 'lophoptera-squammigera', 'athyma-inara', 'satarupa-splendens', 'rapala-pheretima', 'egnasia-ephyrodalis', 'pardomima-distorta', 'attatha-regalis', 'cepora-nadina', 'spoladea-recurvalis', 'helicoverpa-assulta', 'junonia-lemonias', 'pyrausta-panopealis', 'aloa-lactinea', 'hasora-badra', 'celastrina-lavendularis', 'hasora-chromus', 'trypanophora-semihyalina', 'tathorhynchus-exsiccata', 'callerebia-dibangensis', 'nyctemera-coleta', 'actias-selene', 'graphium-evemon', 'cnaphalocrocis-medinalis', 'graphium-nomius', 'leptotes-plinius', 'dysallacta-negatalis', 'papilio-nephelus', 'zizeeria-karsandra', 'macroglossum-mitchellii', 'chlumetia-transversa', 'ancylolomia-taprobanensis', 'symbrenthia-lilaea', 'graphium-macareus', 'zeheba-aureata', 'culladia-suffusella', 'theretra-oldenlandiae', 'ypthima-philomela', 'glyphodes-onychinalis', 'dercas-verhuelli', 'leptomiza-calcearia', 'acropteris-striataria', 'nausinoe-geometralis', 'teliphasa-albifusa', 'olene-mendosa', 'digama-hearseyana', 'bastilla-torrida', 'mangina-syringa', 'creatonotos-gangis', 'ramadasa-pavo', 'poritia-hewitsoni', 'euthalia-phemius', 'epiplema-reticulata', 'penicillaria-jocosatrix', 'utetheisa-lotrix', 'cephonodes-hylas', 'athyma-zeroca', 'zeltus-amasa', 'aporia-agathon', 'colias-croceus', 'euthalia-aconthea', 'cotachena-histricalis', 'pelopidas-conjuncta', 'omphisa-repetitalis', 'bastilla-crameri', 'anisoneura-aluco', 'junonia-iphita', 'koruthaialos-rubecula', 'telinga-malsara', 'lasiommata-schakra', 'cupha-erymanthis', 'sephisa-chandra', 'aglais-urticae', 'conogethes-punctiferalis', 'athyma-selenophora', 'isocentris-filalis', 'spodoptera-exigua', 'aegocera-bimacula', 'diasemiopsis-ramburialis', 'borbo-cinnara', 'eurema-brigitta', 'appias-lyncinda', 'athyma-ranga', 'nomophila-noctuella', 'polyura-athamas', 'phalera-grotei', 'sauris-eupitheciata', 'appias-olferna', 'chrysodeixis-acuta', 'chliaria-othona', 'euclasta-defamatalis', 'rapala-iarbus', 'scobura-isota', 'telicota-bambusae', 'feliniopsis-indistans', 'attacus-atlas', 'symbrenthia-hypselis', 'micronia-aculeata', 'badamia-exclamationis', 'sympis-rufibasis', 'agrius-convolvuli', 'xanthodes-malvae', 'comostola-laesaria', 'eudocima-salaminia', 'ericeia-pertendens', 'graphium-xenocles', 'caleta-decidia', 'anisephyra-ocularia', 'mocis-undata', 'samia-canningii', 'zizina-otis', 'conservula', 'curetis-thetis', 'jamides-bochus', 'polydesma-boarmioides', 'libythea-lepita', 'arhopala-fulla', 'acontia-nitidula', 'pachliopta-aristolochiae', 'lethe-chandica', 'junonia-almana', 'eutectona-machoeralis', 'nyctemera-lacticinia', 'syngamia-latimarginalis', 'conogethes-puntiferalis', 'heliophorus-brahma', 'celastrina-oreana', 'halpe-kusala', 'catapaecilma-major', 'arthroschista-hilaralis', 'arhopala-bazaloides', 'hellula-undalis', 'scobura-parawoolletti', 'antitrygodes-cuneilinea', 'aiteta-rufoflava', 'pseudozizeeria-maha', 'autocharis-fessalis', 'telicota-colon', 'endotricha-mesenterialis', 'helicoverpa-armigera', 'arhopala-perimuta', 'digama-marchalii', 'abisara-neophron', 'astygisa-albopunctata', 'ochus-subvittatus', 'rathinda-amor', 'tarucus-nara', 'zurobata-vacillans', 'leucophlebia-lineate', 'nannoarctia-dentata', 'callyna-jugaria', 'koruthaialos-butleri', 'arhopala-amantes', 'euploea-klugii', 'hypospila-bolinoides', 'colotis-fausta', 'zographetus-rama', 'banisia-myrsusalis', 'pieris-brassicae', 'satarupa-zulla', 'ypthima-chenui', 'notocrypta-paralysos', 'charaxes-marmax', 'atrophaneura-varuna', 'epiplema-himala', 'phycita-steniella', 'erebus-hieroglyphica', 'papilio-polytes', 'udea-ferrugalis', 'theretra-silhetensis-silhetensis', 'mycalesis-patnia', 'pelagodes-quadraria', 'nyctemera-adversata', 'pelopidas-assamensis', 'eustrotia-marginata', 'zamarada-excisa', 'neogurelca-hyas', 'botyodes-flavibasalis', 'macrobrochis-splendens', 'hypolycaena-othona', 'kaniska-canace', 'tanaecia-lepidea', 'utetheisa-pulchelloides', 'callopistria-maillardi', 'eilema-antica', 'graphium-eurypylus', 'acraea-issoria', 'tagiades-menaka', 'thyas-juno', 'acosmeryx-akanshi', 'phalera-combusta', 'filodes-fulvidorsalis', 'theretra-pallicosta', 'ticherra-acte', 'theretra-latreillii-lucasii', 'callerebia-hybrida', 'omiodes-indicata', 'erebus-caprimulgus', 'theretra-clotho', 'pelopidas-subochracea', 'mahathala-ameria', 'pantoporia-hordonia', 'asura-conferta', 'athetis-bremusa', 'homaea-clathrum', 'chadisra-bipars', 'delgamma-pangonia', 'ixias-marianne', 'creatonotos-transiens', 'dierna-strigata', 'lymantria-marginata', 'euploea-radamanthus', 'episteme-adulatrix', 'areas-imperialis', 'perina-nuda', 'virachola-isocrates', 'papilio-memnon', 'appias-libythea', 'eurrhyparodes-tricoloralis', 'theretra-silhetensis', 'jamides-alecto', 'potanthus-pseudomaesa', 'dichromia-pullata', 'vanessa-indica', 'grammodes-stolida', 'hulodes-caranea', 'gastropacha-pardale', 'orphanostigma-abruptalis', 'papilio-polymnestor', 'parasa-herbifera', 'tagiades-litigiosa', 'endotricha-consocia', 'etiella-zinckenella', 'dysphania-militaris', 'scopelodes-venosa', 'chersonesia-intermedia', 'amyna-selenampha', 'chiasmia-nora', 'udaspes-folus', 'argina-argus', 'hypsopygia-mauritialis', 'penthema-lisarda', 'acherontia-lachesis', 'pieris-melete', 'bastilla-arctotaenia', 'phazaca-theclata', 'nephele-hespera', 'amplypterus-panopus', 'lymantria-concolor', 'antheraea-mylitta', 'dodona-eugenes', 'delias-agostina', 'amblypodia-anita', 'hypolycaena-erylus', 'eupterote-mollifera', 'zizula-hylax', 'appias-lalage', 'neptis-yerburyi', 'macroglossum-stellatarum', 'cyclosia-papilionaris', 'halpe-zema', 'glyphodes-stolalis', 'lamprophaia-ablactalis', 'parthenos-sylvia', 'ctenoplusia-albostriata', 'amata-passalis', 'amblychia-hymenaria', 'athyma-kanwa', 'polytela-gloriosae', 'bombyx-mori', 'azanus-jesous', 'notocrypta-feisthamelii', 'parantica-melaneus', 'ophiusa-tirhaca', 'eublemma-anachoresis', 'graphium-chironides', 'jamides-celeno', 'spodoptera-littoralis', 'erebus-albicincta', 'serrodes-inara', 'pleuroptya-iopasalis', 'athetis-thoracica', 'azanus-ubaldus', 'hypena-lividalis', 'dichocrocis-rigidalis', 'arctioblepsis-rubida', 'nygmia-icilia', 'pindara-illibata', 'papilio-alcmenor', 'hebomoia-glaucippe', 'atrophaneura-aidoneus', 'charaxes-aristogiton', 'odontodes-aleuca', 'iton-semamora', 'danaus-chrysippus', 'eudocima-materna', 'protuliocnemis-biplagiata', 'barsine-striata', 'rhagastis-olivacea', 'aglais-rizana', 'leucophlebia-emittens', 'vanessa-cardui', 'noorda-blitealis', 'xenoplia', 'amraica-recursaria', 'chrysodeixis-eriosoma', 'alsophila-nilgirensis', 'theretra-nessus', 'aglaomorpha-plagiata', 'lampides-boeticus', 'artaxa-digramma', 'marumba-dyras', 'pyralis-manihotalis', 'cheritra-freja', 'lethe-verma', 'papilio-machaon', 'eurrhyparodes-bracteolalis', 'everes-diporides', 'problepsis-vulgaris', 'burara-amara', 'parnassius-hardwickii', 'azygophleps-pusilla', 'hyphormides-argentipunctata', 'sinna-extrema', 'surendra-quercetorum', 'remelana-jangala', 'rhinopalpa-polynice', 'delias-eucharis', 'lyclene-undulosa', 'celaenorrhinus-ambareesa', 'marumba-nympha', 'lyssa-zampa', 'syntypistis-fasciata', 'euripus-nyctelius', 'parasa-lepida', 'freyeria-trochylus', 'omiodes-analis', 'fascellina-plagiata', 'colotis-danae', 'pachynoa-sabelialis', 'euploea-algea', 'catopsilia-pyranthe', 'thysanoplusia-orichalcea', 'prosotas-nora', 'leucinodes-orbonalis', 'naranga-diffusa', 'tatobotys-janapalis', 'thaumantis-diores', 'pontia-daplidice', 'pirdana-major', 'neptis-namba', 'matapa-aria', 'rohana-parisatis', 'zographetus-satwa', 'earias-vitella', 'eublemma-dimidialis', 'lethe-mekara', 'spirama-helicina', 'macrobrochis-gigas', 'corma-maculata', 'acraea-violae', 'hyperythra-lutea', 'asota-plaginota', 'ragadia-crisilda', 'euthalia-durga', 'xyleutes-persona', 'pieris-rapae', 'zeuzera-indica', 'homodes-propitia', 'gangara-thyrsis', 'eupterote-lineosa', 'sumalia-daraxa', 'episparis-liturata', 'hypomecis-transcissa', 'aiteta-diurna', 'graphium-sarpedon', 'erebabraxas-metachromata', 'graphium-aristeus', 'taxila-haquinus', 'mycalesis-visala', 'hypolycaena-kina', 'talanga-sexpunctalis', 'catopsilia-pomona', 'ypthima-baldus', 'cydalima-laticostalis', 'orthocabera-ocernaria', 'trabala-vishnou', 'euthalia-anosia', 'ancistroides-nigrita', 'hypochrosis-hyadaria', 'artipe-eryx', 'theretra-castanea', 'netria-viridescens', 'ethope-himachala', 'paectes-subapicalis', 'netria-multispinae', 'westermannia-superba', 'oriens-goloides', 'epiplema', 'dophla-evelina', 'jamides-pura', 'colotis-vestalis', 'miresa-albipuncta', 'mota-massyla', 'vagrans-egista', 'euchromia-polymena', 'euploea-midamus', 'rhodometra-sacraria', 'xanthodes-intersepta', 'hesudra-divisa', 'pareronia-hippia', 'colias-fieldii', 'catochrysops-panormus', 'diplopseustis-periseresalis', 'scirpophaga-nepalensis', 'spirama-indenta', 'cosmophila-flava', 'tyspanodes-linealis', 'graphium-antiphates', 'papilio-clytia', 'arhopala-paramuta', 'telicota-ancilla', 'campylotes-histrionicus', 'mooreana-trichoneura', 'doleschallia-bisaltide', 'papilio-dravidarum', 'hyposidra-talaca', 'curetis-bulis', 'eudocima-cocalus', 'orsotriaena-medus', 'biston-contectaria', 'pygospila-octomaculalis', 'junonia-atlites', 'cupitha-purreea', 'cirrhochrista-fumipalpis', 'byblia-ilithyia', 'matapa-sasivarna', 'marumba-spectabilis', 'actias-maenas', 'heliophorus-epicles', 'satarupa-gopala', 'burara-gomata', 'aedia-acronyctoides', 'conolophia-nigripuncta', 'lethe-sinorix', 'colotis-aurora', 'trilocha-varians', 'bibasis-sena', 'maruca-amboinalis', 'junonia-orithya', 'parantica-pedonga', 'taractrocera-ceramas', 'lassaba-albidaria', 'erionota-torus', 'charaxes-delphis', 'artena-dotata', 'euthrix-laeta', 'thyas-coronata', 'parantica-aglea', 'nevrina-procopia', 'thyas-honesta', 'nacaduba-beroe', 'chersonesia-risa', 'eurema-andersoni', 'bradina-diagonalis', 'parapoynx-fluctuosalis', 'hydrillodes-transversalis', 'chasmina-cadida', 'neptis-pseudovikasi', 'scirpophaga-nivella', 'jamides-elpis', 'ypthima-huebneri', 'kallima-inachus', 'gonodontis-clelia', 'euthalia-monina', 'curetis-dentata', 'mythimna-loreyi', 'anthene-lycaenina', 'acontia-crocata', 'gadirtha-pulchra', 'hypocala-subsatura', 'sphragifera-rejecta', 'chiasmia-emersaria', 'parapoynx-stagnalis', 'catochrysops-strabo', 'sasunaga-tenebrosa', 'eublemma-cochylioides', 'acytolepis-puspa', 'argina-astrea', 'cirrhochrista-pulchellalis', 'bhutanitis-lidderdalii', 'asota-heliconia', 'euploea-core', 'pantoporia-sandaka', 'psilogramma-increta', 'metanastria-aconyta', 'charaxes-psaphon', 'anomis-privata', 'discrepans', 'papilio-mayo', 'chabula-acamasalis', 'bastilla-conficiens', 'lamprosema-tampiusalis', 'mangina-argus', 'agrotis-segetum', 'eurema-blanda', 'freyeria-putli', 'cepora-nerissa', 'suastus-gremius', 'spodoptera-litura', 'eublemma-baccalix', 'heliophorus-sena', 'xandrames-latiferaria', 'discolampa-ethion', 'marumba-indicus', 'herpetogramma-nigricornalis', 'ercheia-cyllaria', 'daphnis-nerii', 'artena-rubida', 'sameodes-cancellalis', 'rapala-varuna', 'amerila-astreus', 'hypocala-deflorata', 'enispe-euthymius', 'parantica-nilgiriensis', 'mycalesis-mineus', 'iraota-timoleon', 'agrotis-biconica', 'papilio-bianor', 'polygrammodes-pectinicornalis', 'araotes-lapithis', 'eoophyla-sejunctalis', 'dyrzela-plagiata', 'paliga-damastesalis', 'macroglossum-belis', 'maliarpha-separatella', 'gerosis-phisara', 'euploea-mulciber', 'pilocrocis-milvinalis', 'dichorragia-nesimachus', 'syntomoides-imaon', 'dysphania-percota', 'euchrysops-cnejus', 'amata-sperbius', 'troides-minos', 'papilio-buddha', 'dudusa-synopla', 'elymnias-malelas', 'problepsis-deliaria', 'hypena-thermesialis', 'oeonistis-entella', 'lestranicus-transpectus', 'polyptychus-dentatus', 'mycalesis-evansii', 'penicillaria-maculata', 'laelia-testacea', 'buzara-onelia', 'brevipecten-captatus', 'pygospila-tyres', 'rajendra-vittata', 'achaea-serva', 'oraesia-emarginata', 'neptis-soma', 'aeromachus-pygmaeus', 'hyblaea-puera', 'gandaca-harina', 'notocrypta-curvifascia', 'pachliopta-hector', 'eterusia-aedea', 'cirrhochrista-spissalis', 'nyctemera-arctata', 'phocoderma-velutina', 'hypolimnas-misippus', 'nausinoe-pueritia', 'macrobrochis-prasena', 'endotricha-melanobasis', 'cania-bilinea', 'asota-plana', 'lymantria-bivittata', 'darpa-pteria', 'terastia-egialealis', 'herpetogramma-cynaralis', 'aemene-taprobanis', 'didyma', 'ophiusa-indistincta', 'orthospila-orissusalis', 'arhopala-pseudocentaurus', 'parotis-nilgirica', 'pericyma-umbrina', 'moduza-procris', 'dysgonia-arcuata', 'hippotion-celerio', 'nebulosa', 'dysgonia-stuposa', 'gonepteryx-rhamni', 'scopula-pulchellata', 'agathia-lycaenaria', 'plusiodonta-coelonota', 'hemithea-tritonaria', 'agrioglypta-eurytusalis', 'plutodes-transmutata', 'coladenia-agni', 'pantoporia-paraka', 'bradina-melanoperas', 'euploea-doubledayi', 'thoressa-gupta', 'pithauria-stramineipennis', 'byasa-polyeuctes', 'ariadne-ariadne', 'fodina-oriolus', 'diaphania-indica', 'acherontia-styx', 'heliothis-irrorata', 'maruca-vitrata', 'curoba-sangarida', 'papilio-protenor', 'hypolimnas-bolina', 'chilades-lajus', 'celaenorrhinus-leucocera', 'erebus-macrops', 'heliophorus-moorei', 'charaxes-kahruba', 'bastilla-simillima', 'caligula-simla', 'pidorus-glaucopis', 'lethe-confusa', 'sarangesa-purendra', 'spindasis-vulcanus', 'catada-prominens', 'cyclodes-omma', 'tarucus-ananda', 'asota-sericea', 'neptis-harita', 'neptis-hylas', 'arsacia-rectalis', 'ixias-pyrene', 'parantica-sita', 'caltoris-kumara', 'arhopala-centaurus', 'symphaedra-nais', 'dichocrocis-zebralis', 'argynnis-hyperbius', 'omphisa-fuscidentalis', 'melanitis-leda', 'lethe-vindhya', 'idaea-macrospila', 'pelopidas-agna', 'melittia-chalciformis', 'antigastra-catalaunalis', 'olepa-ricini', 'hyles-livornica', 'choaspes-benjaminii', 'omphisa-anastomosalis', 'macaretaera-hesperis', 'deudorix-epijarbas', 'asota-producta', 'entomogramma-fautrix', 'hippoton-roseta', 'cethosia-nietneri', 'bastilla-joviana', 'leucinodes-apicalis', 'syllepte-sellalis', 'pagyda-salvalis', 'platyja-umminea', 'glyphodes-bicolor', 'loxura-atymnus', 'spodoptera-mauritia', 'mycalesis-francisca', 'spindasis-elima', 'odontoptilum-angulata', 'haritalodes-derogata', 'parotis-suralis', 'elymnias-hypermnestra', 'pergesa-acteus', 'vindula-erota', 'castalius-rosimon', 'hydriris-ornatalis', 'tajuria-cippus', 'rajendra-perrotteti', 'cethosia-biblis', 'botyodes-patulalis', 'appias-indra', 'tirumala-limniace', 'gerosis-sinica', 'crocidolomia-pavonana', 'achaea-mercatoria', 'rapala-nissa', 'amata-bicincta', 'trigonodes-regalis', 'ypthima-asterope', 'maruca-testulalis', 'orybina-plangonalis', 'pleurona-falcata', 'xanthodes-transversa', 'cydalima-conchylalis', 'aedia-leucomelas', 'blenina-donans', 'herpetogramma-bipunctalis', 'prioneris-sita', 'anuga-lunulata', 'celaenorrhinus-aurivittata', 'gonepteryx-amintha', 'istrugia-disputaria', 'pseudocoladenia-dan', 'mycalesis-perseus', 'arnetta-atkinsoni', 'neopithecops-zalmora', 'graphium-doson', 'sticopthalma-camadeva', 'papilio-krishna', 'rehimena-phrynealis', 'discophora-sondaica', 'orudiza-protheclaria', 'similis', 'abisara-chela', 'arctornis-comma', 'brithys-crini', 'pterothysanus-laticilia', 'cnaphalocrocis-poeyalis', 'trigonodes-disjuncta', 'anticarsia-irrorata', 'sinthusa-nasaka', 'hyarotis-adrastus', 'celaenorrhinus-ruficornis', 'ataboruza-divisa', 'hymenia-perspectalis', 'amyna-axis', 'taraka-hamada', 'delias-belladonna', 'trigonodes-hyppasia', 'thoressa-cerata', 'lymantria-similis', 'spirama-retorta', 'botyodes-asialis', 'sarangesa-dasahara', 'psolos-fuligo', 'lexias-dirtea', 'dacalana-penicilligera', 'heliothis-peltigera', 'graphium-agamemnon', 'asota-caricae', 'ceira-longipennis', 'gonepteryx-amintha-thibetana', 'hippotion-velox', 'lacera-procellosa', 'nagadeba-indecoralis', 'penicillaria-simplex', 'achrosis-rondelaria', 'carriola-ecnomoda', 'loboschiza-koenigiana', 'calyptra-minuticornis', 'appias-lyncida', 'papilio-crino', 'ophiusa-triphaenoides', 'hypopyra-vespertilio', 'cirrochroa-aoris', 'troides-helena', 'cyana-effracta', 'eressa-multigutta', 'maliattha-quadripartita', 'burara-oedipodea', 'eligma-narcissus', 'pseudergolis-wedah', 'synclera-traducalis', 'pannosa', 'oraesia-argyrosigna', 'telinga-malsarida', 'notarcha-quaternalis', 'faunis-canens'}
new_species = { s for s in new_species if len(s.split('-')) > 1 and not re.match(ignore_regex, s) }
new_species -= lepidoptera_classes

# discard possible typo
discarded_species = {s for s in new_species if min([ Levenshtein.distance(s, s2) for s2 in lepidoptera_classes ]) <= 2}
print(f"Discarded: {len(discarded_species)}")
print(discarded_species)
new_species -= discarded_species

new_moth = { s for s in new_species if re.match(moth_regex, s) }
# print(f"\nMoth: {len(new_moth)}")
# print(new_moth)

new_butterfly = { s for s in new_species if re.match(butterfly_regex, s) }
# print(f"\nButterfly: {len(new_butterfly)}")
# print(new_butterfly)

print(f"\nUnidentified: {len(new_species - new_moth - new_butterfly)}")
print(new_species - new_moth - new_butterfly)

Discarded: 24
{'chasmina-cadida', 'sticopthalma-camadeva', 'polygrammodes-pectinicornalis', 'appias-lyncinda', 'leucophlebia-lineate', 'orvasca-subnotota', 'jamides-caeruleus', 'earias-vitella', 'sauris-eupitheciata', 'hippoton-roseta', 'diplopseustis-periseresalis', 'spalgis-epeus', 'ypthima-chenui', 'euthalia-durga', 'homodes-propitia', 'udara-dilecta', 'argina-argus', 'homaloxestis-myloxesta', 'brevipecten-captatus', 'polydesma-boarmioides', 'eutectona-machoeralis', 'conogethes-puntiferalis', 'eurema-andersoni', 'istrugia-disputaria'}

Unidentified: 3
{'indomyrlaea-ferreotincta', 'charltona-kala', 'erebabraxas-metachromata'}


### Current state

In [6]:
species = load_json("species.json")
classes_with_data = {c for c in os.listdir(f"insect-dataset/lepidoptera/data") if not re.match(r"^.*-(early|genera|spp)$", c)}
lepidoptera_classes = set(species['lepidoptera']['moth'] + species['lepidoptera']['butterfly'])
print(f"total = {len(lepidoptera_classes)} = moth:{len(species['lepidoptera']['moth'])} + butterfly:{len(species['lepidoptera']['butterfly'])}")
no_data_classes = lepidoptera_classes - classes_with_data
print(f"with_data = {len(lepidoptera_classes - no_data_classes)}")
print(f"no_data = {len(no_data_classes)}")

total = 4346 = moth:2845 + butterfly:1501
with_data = 4077
no_data = 269
